<div style="color:white;
           display:fill;
           border-radius:25px;
           background-color:Pink;
           font-size:210%;
           font-family:Verdana;
           letter-spacing:0.5px">
<p style="padding: 10px;
          color:white;
          text-align:center;"
          >
       WELCOME TO MY NOTEBOOK
</p>
</div>

<div style="color:white;
           display:fill;
           border-radius:25px;
           background-color:purple;
           font-size:160%;
           font-family:Verdana;
           letter-spacing:0.5px">

<p style="padding: 10px;
          color:white;
          text-align:Center;"
          >
      About Dataset: Employee Dataset


</p>
</div>

![](https://t4.ftcdn.net/jpg/01/42/54/45/360_F_142544579_pmsinNqZBfGfXgP7Tn2EcBkFWlNs6jSm.jpg)

This dataset contains 8 input features which are:
   1. **Education:** Level of Education(Bachelors, Masters and PHD)
   2. **Joining year:** The year of joining
   3. **City:** To which city the employee belongs to
   4. **PaymentTier:** Salary Tiers
   5. **Age:** The age of the Employee
   6. **Gender:** Male and Female
   7. **EverBenched:** Everbenched (yes or no)
   8. **experience in current domain:** Employee experience in terms of years
   
>    And based on these input features, our main goal is to predict whether the Employee **leave or not**.


> Thank you for exploring my notebook. Please take a moment to upvote my notebook. Your support motivates me to keep improving and sharing valuable insights.😊

# Import all the Libraries

In [ ]:
# Import all the necesary Libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,VotingClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,ConfusionMatrixDisplay,classification_report


In [ ]:
# Read the dataset
dataframe= pd.read_csv("/kaggle/input/employee-dataset/Employee.csv")
dataframe.head(10)

In [ ]:
# Lets check the shape of the data
dataframe.shape

In [ ]:
# Lets check the datatype of each columns
dataframe.info()

In [ ]:
# Lets describe the dataset
dataframe.describe()

In [ ]:
# To check Is there any null values in the dataset
dataframe.isna().sum()

> Here we can see that there is no null values in the dataset.

In [ ]:
# To check Is there any duplicate value in the dataset
dataframe.duplicated().sum()

In [ ]:
# Remove the duplicate values in the dataset
dataframe.drop_duplicates(inplace=True)

In [ ]:
# Shape of dataframe after removing the duplicate values
dataframe.shape

# Lets check the unique values in the categorical columns

In [ ]:
dataframe["Education"].unique()

In [ ]:
dataframe["City"].unique()

In [ ]:
dataframe["Gender"].unique()

In [ ]:
dataframe["EverBenched"].unique()

# Exploratory Data Analysis

> # Univariate Analysis

In [ ]:
# Check the count of target variable
dataframe["LeaveOrNot"].value_counts()

In [ ]:
plt.figure(figsize=(5,4))
sns.countplot(data=dataframe, x=dataframe["LeaveOrNot"], color="red")
plt.show()

In [ ]:
dataframe["Education"].value_counts()

In [ ]:
sns.countplot(data=dataframe, x= dataframe["Education"], hue="LeaveOrNot", color="red")

> Here we can notice that almost half of the Masters Student Leave the job, whereas the more number of Bachelor's and PHD student stick to their jobs.

In [ ]:
dataframe["Gender"].value_counts()

In [ ]:
sns.countplot(data=dataframe, x= dataframe["Gender"], hue="LeaveOrNot", color="blue")

> Here we can see that half of the female leave their jobs,whereas the more no. of male candidates prefer to do the jobs.

In [ ]:
dataframe["City"].value_counts()

In [ ]:
sns.countplot(data=dataframe, x= dataframe["City"], hue="LeaveOrNot")

> This barchart indicates that in Pune City half of the people leave their jobs, whereas the  Banglore and New Delhi are the cities where people are willing to do the jobs.

In [ ]:
dataframe["EverBenched"].value_counts()

In [ ]:
sns.countplot(data=dataframe, x= dataframe["EverBenched"], hue="LeaveOrNot", color="green")

In [ ]:
sns.histplot(data=dataframe, x=dataframe["JoiningYear"],hue="LeaveOrNot", kde=True, bins=6, multiple="stack")

> Here we can see that small number of people leave the jobs during the year 2012-2013, 2014-2015 and 2016-2017 as compared to the rest of years.

In [ ]:
sns.histplot(data=dataframe, x=dataframe["PaymentTier"],hue="LeaveOrNot", kde=True, bins=3, multiple="stack")

> Here we can clearly see that when the salary range is in between the 2.20-3.00, more people willing to stay in the company.

> 

In [ ]:
sns.histplot(data=dataframe, x=dataframe["Age"],hue="LeaveOrNot", kde=True, bins=20, multiple="stack")

> From this representation, we observe that the more number of people, who ranges between the age of 22-26, prefer to change the jobs, whereas in the rest age group we see that more people prefer to stick to their jobs.

In [ ]:
sns.histplot(data=dataframe, x=dataframe["ExperienceInCurrentDomain"],hue="LeaveOrNot", kde=True, bins=7, multiple="stack")

> # Bivariate Analsysis

# Label Encoding for Categorical Columns

In [ ]:
le=LabelEncoder()
dataframe["Education"]= le.fit_transform(dataframe["Education"])
dataframe["City"]=le.fit_transform(dataframe["City"])
dataframe["Gender"]=le.fit_transform(dataframe["Gender"])
dataframe["EverBenched"]=le.fit_transform(dataframe["EverBenched"])

In [ ]:
# Lets plot the correlation matrix 
plt.figure(figsize=(8,5))
sns.heatmap(dataframe.corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.show()

> 1. Here we can see that LeaveOrNot are correlated with variables joining year and city with the values of 0.15 and 0.13.
2. Joining year and City are correlated with the Edcation with the value of 0.11.
3. Gender is correalted with the PaymentTier with the value of 0.17.

# Split the Data into Train and Test Set

In [ ]:
# Lets Split the dataset 
def train_test_split_data(dataframe, target, random_state, test_size):
    x_train,y_train,x_test,y_test= train_test_split(dataframe.drop([target], axis=1),
                                                    dataframe[target],
                                                    random_state=random_state,
                                                    test_size=0.3,
                                                    stratify=dataframe[target])
    return x_train, x_test, y_train,y_test

In [ ]:
x_train, y_train, x_test, y_test=  train_test_split_data(dataframe, target="LeaveOrNot", random_state=42, test_size=0.3)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

# Lets balance the dataset using Smote Oversampling Technique

In [ ]:
# Lets balance the dataset using smote technique
sm= SMOTE(sampling_strategy="minority", random_state=42)
x_train_smote, y_train_smote= sm.fit_resample(x_train, y_train)

In [ ]:
x_train_smote.shape, y_train_smote.shape

# Lets do the Modelling

> # Random Forest Classifier

In [ ]:
rf=RandomForestClassifier()

hyper_params = {"max_features": [1,3,10],
                "min_samples_split":[2,3,10],
                "min_samples_leaf":[1,3,10],
                "n_estimators":[100,300, 400, 500, 600],
                "criterion":["gini"]}
    
model=GridSearchCV(rf, hyper_params, cv=5, scoring="f1_macro", n_jobs=-1)
model.fit(x_train_smote,y_train_smote)
y_pred= model.predict(x_test)
print(model.best_estimator_)

print("----------------------")

accuracy_scores=[]
f1_scores=[]
precision_scores=[]
recall_scores=[]


accuracy_scores.append(accuracy_score(y_pred, y_test))
f1_scores.append(f1_score(y_pred, y_test))
precision_scores.append(precision_score(y_pred, y_test))
recall_scores.append(recall_score(y_pred, y_test))



# Print the Results
print(f"Accuracy:{accuracy_scores}")
print(f"F1-Score:{f1_scores}")
print(f"Precision:{precision_scores}") 
print(f"Recall:{recall_scores}")

print("-------------------------------")
print("Classifiaction Reoprt")
print("-------------------------------")
print(classification_report(y_test,y_pred,digits=3))
print("Confusion_Matrix")
ConfusionMatrixDisplay.from_predictions(y_test,y_pred,cmap="Reds")
plt.show()

> # Gradient Boosting Classifier

In [ ]:
gbc_param_grid = {
                      "learning_rate": [0.05, 0.1, 0.02],
                      "n_estimators": [100,200,300,400,500],
                      "min_samples_split": [2,3,10],
                      "min_samples_leaf": [1,3,10]
                      }

gbc=GradientBoostingClassifier()
gbc_model= GridSearchCV(gbc, gbc_param_grid, cv=5, scoring="f1_macro", n_jobs=-1 )
gbc_model.fit(x_train_smote,y_train_smote)
y_pred= gbc_model.predict(x_test)
print(gbc_model.best_estimator_)

accuracy_scores=[]
f1_scores=[]
precision_scores=[]
recall_scores=[]


accuracy_scores.append(accuracy_score(y_pred, y_test))
f1_scores.append(f1_score(y_pred, y_test))
precision_scores.append(precision_score(y_pred, y_test))
recall_scores.append(recall_score(y_pred, y_test))



# Print the Results
print(f"Accuracy:{accuracy_scores}")
print(f"F1-Score:{f1_scores}")
print(f"Precision:{precision_scores}") 
print(f"Recall:{recall_scores}")

print("-------------------------------")
print("Classifiaction Reoprt")
print("-------------------------------")
print(classification_report(y_test,y_pred,digits=3))
print("Confusion_Matrix")
ConfusionMatrixDisplay.from_predictions(y_test,y_pred,cmap="Greens")
plt.show()

> # Voting Classifier

In [ ]:
gbc=GradientBoostingClassifier()
gbc1=GradientBoostingClassifier(learning_rate=0.05, min_samples_split=3,n_estimators=500)
rf=RandomForestClassifier(max_features=3, min_samples_leaf=3, min_samples_split=3)


voting_model= VotingClassifier(estimators=[('gradient_bosting1', gbc),('gradient_bosting2', gbc), ('gradient_bosting3', gbc), ('gradient_bosting4',gbc),('gradient_bosting5',gbc)], voting='hard', n_jobs=-1)
voting_model.fit(x_train_smote, y_train_smote)
y_pred=voting_model.predict(x_test)


accuracy_scores=[]
f1_scores=[]
precision_scores=[]
recall_scores=[]


accuracy_scores.append(accuracy_score(y_pred, y_test))
f1_scores.append(f1_score(y_pred, y_test))
precision_scores.append(precision_score(y_pred, y_test))
recall_scores.append(recall_score(y_pred, y_test))



# Print the Results
print(f"Accuracy:{accuracy_scores}")
print(f"F1-Score:{f1_scores}")
print(f"Precision:{precision_scores}") 
print(f"Recall:{recall_scores}")

print("-------------------------------")
print("Classifiaction Reoprt")
print("-------------------------------")
print(classification_report(y_test,y_pred,digits=3))
print("Confusion_Matrix")
ConfusionMatrixDisplay.from_predictions(y_test,y_pred,cmap="Purples")
plt.show()

# Visualize the Final Results of Voting Classifier

In [ ]:
model=["Voting_Classifier"]
result_df= pd.DataFrame({"Accuracy":accuracy_scores, "F1_score":f1_scores, "Precision":precision_scores, "Recall": recall_scores}, index=model)
result_df=result_df.T.sort_values(by="Voting_Classifier", ascending=False)
result_df

In [ ]:
result_df.plot(kind="bar", figsize=(8,6), color="green").legend(bbox_to_anchor=(1.3,1));

<div style="color:white;
           display:fill;
           border-radius:25px;
           background-color:pink;
           font-size:160%;
           font-family:Verdana;
           letter-spacing:0.5px">

<p style="padding: 10px;
          color:white;
          text-align:Center;"
          >
     Conclusion


</p>
</div>

>  1. By exploring the data we came to know that the distribution of dataset is imbalanced, so we used the SMOTE oversampling technique to balance the class distribution and then train the machine learning models.
 2. We notice that the Voting Classifier model performs best in terms of Accuracy that is 79%  and with Recall 75%.